<a href="https://colab.research.google.com/github/rafavidal1709/mack-ai-plataforma/blob/main/MackAI_Instanciar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PROJECT_ID = "mackai-468422"
ROOT_KEY = "aA!12345"
DB_NAME = "mackai"
REGION="us-central1"
REPO="cloud-run"

# CloudSQL

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip -q install cloud-sql-python-connector
!pip -q install pymysql sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 620.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 801.7 kB/s eta 0:00:00


In [ ]:
from googleapiclient.discovery import build

def print_instances():
  sqladmin = build("sqladmin", "v1beta4", cache_discovery=False)
  req = sqladmin.instances().list(project=PROJECT_ID)
  resp = req.execute()

  print("===== Instancias: =====")

  for inst in resp.get("items", []):
      print(f"Instância: {inst['name']} | Tipo: {inst['databaseVersion']} | Regiao: {inst['region']} | IPs: {inst.get('ipAddresses', [])}")

  print("=======================")

print_instances()

===== Instancias: =====


In [ ]:
!gcloud sql instances create {DB_NAME} \
  --database-version=MYSQL_8_0 \
  --tier=db-f1-micro \
  --region=us-central1 \
  --root-password={ROOT_KEY} \
  --project={PROJECT_ID}


Created [https://sqladmin.googleapis.com/sql/v1beta4/projects/mackai-468422/instances/mackai].
NAME    DATABASE_VERSION  LOCATION       TIER         PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
mackai  MYSQL_8_0         us-central1-c  db-f1-micro  34.63.216.169    -                RUNNABLE


In [ ]:
!gcloud sql instances patch {DB_NAME} \
  --project={PROJECT_ID} \
  --assign-ip


The following message will be used for the patch API method.
{"name": "mackai", "project": "mackai-468422", "settings": {"ipConfiguration": {"ipv4Enabled": true}}}
Updated [https://sqladmin.googleapis.com/sql/v1beta4/projects/mackai-468422/instances/mackai].


In [ ]:
print_instances()

In [ ]:
!gcloud sql instances patch {DB_NAME} \
  --project={PROJECT_ID} \
  --authorized-networks=0.0.0.0/0 \
  --quiet


The following message will be used for the patch API method.
{"name": "mackai", "project": "mackai-468422", "settings": {"ipConfiguration": {"authorizedNetworks": [{"value": "0.0.0.0/0"}]}}}
Updated [https://sqladmin.googleapis.com/sql/v1beta4/projects/mackai-468422/instances/mackai].


In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy
from sqlalchemy import text, inspect
import requests  # <— novo

CONNECTION_NAME = f"{PROJECT_ID}:us-central1:{DB_NAME}"  # NOTE: deve ser PROJECT:REGION:INSTANCE

connector = Connector()

def getconn_sys():
    return connector.connect(CONNECTION_NAME, "pymysql",
                             user="root", password=ROOT_KEY, db="mysql")

engine_sys = sqlalchemy.create_engine("mysql+pymysql://", creator=getconn_sys)

def getconn_app():
    return connector.connect(CONNECTION_NAME, "pymysql",
                             user="root", password=ROOT_KEY, db=DB_NAME)

engine = sqlalchemy.create_engine("mysql+pymysql://", creator=getconn_app)

with engine_sys.begin() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {DB_NAME} "
                      "CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci"))

# --- lê o arquivo .sql diretamente do GitHub (raw) ---
RAW_SQL_URL = "https://raw.githubusercontent.com/rafavidal1709/mack-ai-plataforma/main/ecovir28_mackai.sql"
resp = requests.get(RAW_SQL_URL, timeout=60)
resp.raise_for_status()
sql_script = resp.text.replace("\r\n", "\n").lstrip("\ufeff")  # normaliza EOL e remove BOM

# (opcional) se seu MySQL for 5.7, troque o collation 0900 por um compatível:
# sql_script = sql_script.replace("utf8mb4_0900_ai_ci", "utf8mb4_unicode_ci")

# divide em comandos (simples: por ;)
commands = [cmd.strip() for cmd in sql_script.split(";") if cmd.strip()]

with engine.begin() as conn:   # begin() já faz commit
    for cmd in commands:
        conn.execute(text(cmd))

insp = inspect(engine)
tables = insp.get_table_names()
print("Tabelas:", tables, "\n")

with engine.connect() as conn:
    for tbl in tables:
        print(f"\n===== {tbl} =====")
        result = conn.execute(text(f"SELECT * FROM `{tbl}` LIMIT 10"))  # LIMIT para não trazer tudo
        rows = result.fetchall()
        if not rows:
            print("(vazia)")
        else:
            for row in rows:
                print(row)


Tabelas: ['encontro', 'grupo', 'periodo'] 


===== encontro =====
(1, 1, 2, datetime.datetime(2024, 8, 12, 21, 0), 'A história do Perceptron', 'Visão geral sobre o Perceptron, sua história e importância no desenvolvimento do campo do aprendizado de máquina. ', 'rc9cFq8M-Ys', datetime.datetime(2025, 8, 12, 15, 58, 44), datetime.datetime(2025, 8, 12, 19, 23, 30))
(2, 1, 2, datetime.datetime(2024, 8, 19, 21, 0), 'Treinando a Rede', 'Para treinar a rede usa-se o algorítimo de backpropagation', 'rc9cFq8M-Ys', datetime.datetime(2025, 8, 12, 15, 58, 44), datetime.datetime(2025, 8, 12, 19, 23, 33))

===== grupo =====
(1, 'Aprendizagem de Máquina', datetime.datetime(2025, 8, 12, 12, 53, 33), datetime.datetime(2025, 8, 12, 12, 53, 33))
(2, 'Processamento de Linguagem Natural', datetime.datetime(2025, 8, 12, 12, 53, 33), datetime.datetime(2025, 8, 12, 12, 53, 33))
(3, 'Ética', datetime.datetime(2025, 8, 12, 12, 53, 33), datetime.datetime(2025, 8, 12, 12, 53, 33))
(4, 'Projetos', datetime.datetime

In [ ]:
ip = !gcloud sql instances describe {DB_NAME} \
        --project={PROJECT_ID} \
        --format="value(ipAddresses.ipAddress)"

PRIMARY_ADDRESS = ip[0]
print("HOST =", PRIMARY_ADDRESS)
print("DBNAME =", DB_NAME)
print("USER =", "root")
print("PASS =", ROOT_KEY)

HOST = 34.63.216.169
DBNAME = mackai
USER = root
PASS = aA!12345


In [ ]:
!gcloud sql instances patch {DB_NAME} \
  --project={PROJECT_ID} \
  --no-deletion-protection

!gcloud sql instances delete {DB_NAME} --project={PROJECT_ID} --quiet

ERROR: (gcloud.sql.instances.patch) unrecognized arguments:
  --
  quiet
  To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS
Deleted [https://sqladmin.googleapis.com/sql/v1beta4/projects/mackai-468422/instances/mackai].


# Instanciar CloudRUN

In [ ]:
import os
os.environ.update({
  "PROJECT_ID": PROJECT_ID,
  "REGION": REGION,
  "REPO": "cloud-run",
  "SERVICE": "php-service",
  "PRIMARY_ADDRESS": PRIMARY_ADDRESS,  # ou defina DB_HOST direto
  "DB_NAME": DB_NAME,
  "DB_USER": "root",
  "DB_PASS": ROOT_KEY
})


In [ ]:
%%bash
set -euo pipefail

#####################################
# VARS GLOBAIS (já definidas via os.environ)
#####################################
: "${PROJECT_ID:?Defina PROJECT_ID antes}"
: "${REGION:?Defina REGION antes}"
: "${REPO:?Defina REPO antes}"              # ex: cloud-run
: "${SERVICE:?Defina SERVICE antes}"        # ex: php-service

# DB_HOST pode vir de DB_HOST ou PRIMARY_ADDRESS
DB_HOST="${DB_HOST:-${PRIMARY_ADDRESS:-}}"
: "${DB_HOST:?Defina DB_HOST ou PRIMARY_ADDRESS antes}"
: "${DB_NAME:?Defina DB_NAME antes}"
: "${DB_USER:?Defina DB_USER antes}"
: "${DB_PASS:?Defina DB_PASS antes}"

# Tag única p/ nova revisão
IMAGE_TAG="${IMAGE_TAG:-$(date +%Y%m%d-%H%M%S)}"
IMAGE_URI="${REGION}-docker.pkg.dev/${PROJECT_ID}/${REPO}/${SERVICE}:${IMAGE_TAG}"

# URL do seu repo (ajuste se for outro)
GIT_URL="${GIT_URL:-https://github.com/rafavidal1709/mack-ai-plataforma.git}"

echo "IMAGE_URI=$IMAGE_URI"

#####################################
# CONFIG GCP
#####################################
gcloud config set project "$PROJECT_ID" >/dev/null
gcloud config set run/region "$REGION" >/dev/null
gcloud services enable artifactregistry.googleapis.com run.googleapis.com cloudbuild.googleapis.com -q
gcloud artifacts repositories create "$REPO" \
  --repository-format=docker \
  --location="$REGION" \
  --description="Repo para Cloud Run" || true
gcloud auth configure-docker "${REGION}-docker.pkg.dev" -q

#####################################
# SYNC DO CÓDIGO (GitHub → pasta local)
#####################################
if [ -d mack-ai-plataforma ]; then
  pushd mack-ai-plataforma >/dev/null
  git fetch --all
  git reset --hard origin/main
  popd >/dev/null
else
  git clone "$GIT_URL"
fi

#####################################
# DOCKERFILE (garante pdo_mysql)
#####################################
cd mack-ai-plataforma/imagem

# Cria/substitui um Dockerfile mínimo com pdo_mysql
cat > Dockerfile <<'EOF'
FROM php:8.3-cli

# Instala o driver PDO para MySQL
RUN docker-php-ext-install pdo pdo_mysql

WORKDIR /app
COPY . /app

# Cloud Run usa a variável PORT (padrão 8080)
ENV PORT=8080
EXPOSE 8080

# Servidor embutido do PHP
CMD ["sh","-c","php -S 0.0.0.0:${PORT} /app/index.php"]
EOF

echo "Dockerfile pronto:"
head -n 20 Dockerfile

#####################################
# BUILD + DEPLOY (env-vars separadas)
#####################################
gcloud builds submit . --tag "$IMAGE_URI"

gcloud run deploy "$SERVICE" \
  --image "$IMAGE_URI" \
  --region "$REGION" \
  --allow-unauthenticated \
  --set-env-vars DB_HOST="$DB_HOST" \
  --set-env-vars DB_NAME="$DB_NAME" \
  --set-env-vars DB_USER="$DB_USER" \
  --set-env-vars DB_PASS="$DB_PASS"

# Mostrar URL e testar rapidamente
URL="$(gcloud run services describe "$SERVICE" --region "$REGION" --format='value(status.url)')"
echo "URL: $URL"
curl -si "$URL" | head -n 20 || true


IMAGE_URI=us-central1-docker.pkg.dev/mackai-468422/cloud-run/php-service:20250823-011206
Fetching origin
HEAD is now at c661402 Update index.php
Dockerfile pronto:
FROM php:8.3-cli

# Instala o driver PDO para MySQL
RUN docker-php-ext-install pdo pdo_mysql

WORKDIR /app
COPY . /app

# Cloud Run usa a variável PORT (padrão 8080)
ENV PORT=8080
EXPOSE 8080

# Servidor embutido do PHP
CMD ["sh","-c","php -S 0.0.0.0:${PORT} /app/index.php"]
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6f4cd1c6-8a3b-49e1-b7fa-5e6b33f28815"

FETCHSOURCE
Fetching storage object: gs://mackai-468422_cloudbuild/source/1755911550.394519-7ca7194d620d4d6eae03cc942ac21412.tgz#1755911551658025
Copying gs://mackai-468422_cloudbuild/source/1755911550.394519-7ca7194d620d4d6eae03cc942ac21412.tgz#1755911551658025...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2 KiB.
BUILD
Already have image (with di

Updated property [core/project].
Updated property [run/region].
Operation "operations/acat.p2-576668051133-9a24341e-a8a3-4580-bae0-07fbe8e54193" finished successfully.
Create request issued for: [cloud-run]
Waiting for operation [projects/mackai-468422/locations/us-central1/operations/d3e85c4d-2a55-4f48-b61f-046292123f78] to complete...
..............done.
Created repository [cloud-run].
`docker` and `docker-credential-gcloud` need to be in the same PATH in order to work correctly together.
gcloud's Docker credential helper can be configured but it will not work until this is corrected.

{
  "credHelpers": {
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.
Creating temporary archive of 3 file(s) totalling 10.0 KiB before compression.
Uploading tarball of [.] to [gs://mackai-468422_cloudbuild/source/1755911550.394519-7ca7194d620d4d6eae03cc942ac21412.tgz]
Created [https://cloudbuild

In [ ]:
# Excluir repo de imagens
!gcloud artifacts repositories delete cloud-run --location=us-central1 --quiet

# Excluir também o serviço se ainda existir
!gcloud run services delete php-service --region=us-central1 --quiet


Delete request issued for: [cloud-run]
Deleted repository [cloud-run].
Deleted service [php-service].


# Atualizar index.php